In [4]:
!pip install git+https://github.com/rwalk/gsdmm.git

  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-kpncet29
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-kpncet29
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done


In [1]:
import warnings
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib.pyplot import figure
import ast
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from wordcloud_fa import WordCloudFa
from matplotlib.patches import Rectangle
from matplotlib.ticker import FuncFormatter
import time

from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

from sklearn.model_selection import GridSearchCV

import random
import math

# depend on pyLDAvis version import one of the followings
import pyLDAvis.gensim_models
# import pyLDAvis.gensim

from gsdmm import MovieGroupProcess

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

/mnt/c/Users/Parnian/Desktop/ELEARN/Research/v_env/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
base_path = "../remote-work-twitter-analysis/data/"

In [3]:
input_file = base_path + 'prep1_tweets.csv'
tqdm.pandas()
df = pd.read_csv(input_file)
df

/tmp/ipykernel_23386/1282473635.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


,conversation_id,retweets_count,mentions,created_at,likes_count,in_reply_to_user_id,_id,source,user_id,in_reply_to_status_id,...,quoted_status_id,cashtags,urls,tweet,hashtags,tweet_tokens,lemmatized_tweet,lemmatized_tweet_token,tweet_size,lemmatized_tweet_size
0,1375557622410149894,2,NaN,2021-03-26 21:23:21+00:00,111,5.720514e+08,1375559020451356674,Twitter for iPhone,572051429,1.375558e+18,...,NaN,NaN,NaN,گزارش حقوقش میدونم سالی ۸۵ همسرش دانشجو دکترا ...,[],"['گزارش', 'حقوقش', 'میدونم', 'سالی', '۸۵', 'هم...",گزارش حقوق میدونم ۸۵ همسر دانشجو دکترا شرکت اح...,"['گزارش', 'حقوق', 'میدونم', '۸۵', 'همسر', 'دان...",32,30
1,1375559420176908296,3,NaN,2021-03-26 21:24:57+00:00,163,NaN,1375559420176908296,Twitter for iPhone,812764546012246017,NaN,...,NaN,NaN,NaN,اینایی سفر مهمونی پروتکلی رعایت نکردن سرکار خط...,[],"['اینایی', 'سفر', 'مهمونی', 'پروتکلی', 'رعایت'...",اینایی سفر مهمونی پروتکل رعایت نکردن سرکار خطر...,"['اینایی', 'سفر', 'مهمونی', 'پروتکل', 'رعایت',...",11,11
2,1383828354197979136,2,NaN,2021-04-18 17:02:44+00:00,37,NaN,1383828354197979136,Twitter for Android,985101690536898565,NaN,...,NaN,NaN,NaN,خسته دورکاری شهرهای سروصدا شب ازصبح آزاردهنده ...,[],"['خسته', 'دورکاری', 'شهرهای', 'سروصدا', 'شب', ...",خسته دورکاری شهر سروصدا شب ازصبح آزاردهنده تره...,"['خسته', 'دورکاری', 'شهر', 'سروصدا', 'شب', 'از...",26,26
3,1383840315031781385,0,NaN,2021-04-18 17:50:16+00:00,11,NaN,1383840315031781385,Twitter Web App,310871673,NaN,...,1.383657e+18,NaN,NaN,آبان ۹۸ دورکار,[],"['آبان', '۹۸', 'دورکار']",آبان ۹۸ دورکار,"['آبان', '۹۸', 'دورکار']",3,3
4,1383820625354002439,1,['15782990'],2021-04-18 17:31:42+00:00,9,1.578299e+07,1383835643239747593,Twitter Web App,1271095835028307969,1.383821e+18,...,NaN,NaN,NaN,حتمن میرفتم شهری اقوام دوستام نزدیکتر خونه اجا...,[],"['حتمن', 'میرفتم', 'شهری', 'اقوام', 'دوستام', ...",حتمن میرفتم شهر اقوام دوستام خونه اجازه میکردم...,"['حتمن', 'میرفتم', 'شهر', 'اقوام', 'دوستام', '...",16,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45354,1434127275587428352,0,['1273546506012897281'],2021-09-04 12:46:01+00:00,3,1.273547e+18,1434135662396018689,Twitter for Android,1260775960267444225,1.434127e+18,...,NaN,NaN,NaN,لجم میگیره کثافت دورکاری سخت کار اداره ست ذهنا...,[],"['لجم', 'میگیره', 'کثافت', 'دورکاری', 'سخت', '...",لج میگیره کثافت دورکاری سخت کار اداره ست ذهنای...,"['لج', 'میگیره', 'کثافت', 'دورکاری', 'سخت', 'ک...",15,15
45355,1434127961943326723,0,['1251428296589488129'],2021-09-04 12:21:54+00:00,2,1.251428e+18,1434129593036939267,Twitter for Android,1214804130096668672,1.434128e+18,...,NaN,NaN,NaN,اطلاع ماهه رزومه فرستادم جواب ندادن آگهی‌هاشون...,[],"['اطلاع', 'ماهه', 'رزومه', 'فرستادم', 'جواب', ...",اطلاع ماهه رزومه فرستاد#فرست جواب ندادن آگهی‌ه...,"['اطلاع', 'ماهه', 'رزومه', 'فرستاد#فرست', 'جوا...",9,8
45356,1434138057071112194,0,NaN,2021-09-04 12:55:32+00:00,4,NaN,1434138057071112194,Twitter for iPhone,1147481863138811905,NaN,...,NaN,NaN,NaN,خواهرم کار استخدام بشه دورکاری الآن نخواهم_تون...,[],"['خواهرم', 'کار', 'استخدام', 'بشه', 'دورکاری',...",خواهر کار استخدام بشه دورکاری الآن نخواهم_تونس...,"['خواهر', 'کار', 'استخدام', 'بشه', 'دورکاری', ...",15,15
45357,1434133064750338048,0,NaN,2021-09-04 12:35:42+00:00,18,NaN,1434133064750338048,Twitter for Android,1106832761216815104,NaN,...,NaN,NaN,NaN,دورکاری,[],['دورکاری'],دورکاری,['دورکاری'],1,1


In [4]:
# preprocessed = []
# for record in tqdm(reader.iterrows()):
#     record['tweet_tokens'] = record['tweet_tokens'].apply(lambda t: ast.literal_eval(t))
#     record['lemmatized_tweet_token'] = record['lemmatized_tweet_token'].apply(lambda t: ast.literal_eval(t))
#     preprocessed.append(record)

In [5]:
token_list_lemm = []
token_list = []
for ind in df.index:
    # print(type(df['lemmatized_tweet_token'][ind]))
    token_list_lemm += [ast.literal_eval(df['lemmatized_tweet_token'][ind])]
    token_list += [ast.literal_eval(df['tweet_tokens'][ind])]

In [6]:
def get_dictionary(tokens):
    dictionary = corpora.Dictionary(tokens)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)    
    return dictionary

In [7]:
sample_dict = get_dictionary(token_list)
count = 0
for k, v in sample_dict.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 ادامه
1 براش
2 تمدید
3 جالب
4 حقوق
5 حقوقش


## BOW

In [8]:
def get_bow_corpus(dictionary, tokens):
    return [dictionary.doc2bow(doc) for doc in tokens]

### N-Lem

In [26]:
from gensim.models.ldamodel import LdaModel

all_tokens = token_list
all_dictionary = get_dictionary(all_tokens)
all_corpus = get_bow_corpus(all_dictionary, all_tokens)
lda_topic_model = gensim.models.LdaMulticore(tqdm(all_corpus), num_topics=5, id2word=all_dictionary, passes=10, alpha=0.1)

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 25478.62it/s]


In [27]:
from pprint import pprint
pprint(lda_topic_model.print_topics())
docs_lda = lda_topic_model[all_corpus]

[(0,
  '0.034*"کار" + 0.018*"پروژه" + 0.014*"دورکار" + 0.012*"بازارکار" + '
  '0.011*"استخدام" + 0.011*"برنامه" + 0.010*"انجام" + 0.009*"صورت" + '
  '0.009*"\u2063" + 0.007*"ارسال"'),
 (1,
  '0.056*"کار" + 0.029*"خونه" + 0.017*"دورکار" + 0.010*"میشه" + 0.009*"شرکت" + '
  '0.008*"😂" + 0.007*"صبح" + 0.007*"کاری" + 0.007*"انجام" + 0.006*"سرکار"'),
 (2,
  '0.039*"کار" + 0.020*"دورکار" + 0.018*"شرکت" + 0.012*"ریموت" + 0.009*"میشه" '
  '+ 0.007*"کرونا" + 0.005*"واکسن" + 0.005*"کنن" + 0.005*"حقوق" + '
  '0.005*"حضوری"'),
 (3,
  '0.037*"کرونا" + 0.020*"کار" + 0.010*"خانه" + 0.009*"تهران" + 0.009*"ایران" '
  '+ 0.008*"شرایط" + 0.007*"کارمندان" + 0.007*"دولت" + 0.007*"دورکار" + '
  '0.006*"قرنطینه"'),
 (4,
  '0.076*"کار" + 0.062*"منزل" + 0.057*"چاپ" + 0.040*"خانگی" + 0.040*"شغل" + '
  '0.026*"سیلک" + 0.025*"دستگاه" + 0.017*"خانه" + 0.016*"پردرآمد" + '
  '0.013*"نایلون"')]


In [28]:
coherence_lda = CoherenceModel(model=lda_topic_model, texts=all_tokens, dictionary=all_dictionary, coherence='c_v')
coherence_value = coherence_lda.get_coherence()
coherence_value

0.5145017169926824

### Lem

In [12]:
all_tokens = token_list_lemm
all_dictionary = get_dictionary(all_tokens)
all_corpus = get_bow_corpus(all_dictionary, all_tokens)
lda_topic_model = gensim.models.LdaMulticore(tqdm(all_corpus), num_topics=4, id2word=all_dictionary, passes=10, alpha=0.1)

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 25519.01it/s]


In [13]:
pprint(lda_topic_model.print_topics())
docs_lda = lda_topic_model[all_corpus]

[(0,
  '0.070*"کار" + 0.021*"خونه" + 0.012*"شرکت" + 0.012*"ریموت" + 0.011*"میشه" + '
  '0.010*"کاشت#کار" + 0.009*"دورکار" + 0.007*"کرونا" + 0.006*"گرفت#گیر" + '
  '0.006*"زد#زن"'),
 (1,
  '0.091*"کار" + 0.066*"چاپ" + 0.066*"منزل" + 0.051*"شغل" + 0.046*"خانگی" + '
  '0.031*"خانه" + 0.029*"سیلک" + 0.028*"دستگاه" + 0.023*"زن" + '
  '0.019*"پردرآمد"'),
 (2,
  '0.031*"دورکار" + 0.020*"کار" + 0.019*"پروژه" + 0.011*"بازارکار" + '
  '0.011*"شرکت" + 0.011*"برنامه" + 0.010*"استخدام" + 0.009*"همکار" + '
  '0.009*"انجام" + 0.009*"صورت"'),
 (3,
  '0.024*"کرونا" + 0.015*"کار" + 0.014*"کارمند" + 0.014*"تهران" + 0.011*"شرکت" '
  '+ 0.010*"دورکار" + 0.009*"حضور" + 0.009*"دولت" + 0.008*"تعطیل" + '
  '0.008*"شرایط"')]


In [14]:
coherence_model_lda = CoherenceModel(model=lda_topic_model, texts=all_tokens, dictionary=all_dictionary, coherence='c_v')
coherence_value = coherence_model_lda.get_coherence()
coherence_value

0.549433686919246

## TF-IDF

In [15]:
def get_tf_idf_corpus(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    return tfidf[bow_corpus]

### N-Lem

In [16]:
all_tokens = token_list
all_dictionary = get_dictionary(all_tokens)
bow_corpus = get_bow_corpus(all_dictionary, all_tokens)
corpus_tf_idf = get_tf_idf_corpus(bow_corpus)
lda_topic_model = gensim.models.LdaMulticore(tqdm(corpus_tf_idf), num_topics=4, id2word=all_dictionary, passes=10, alpha=0.1)

100%|███████████████████████████████████████████████████████████████████████████| 45359/45359 [00:04<00:00, 9748.81it/s]


In [17]:
pprint(lda_topic_model.print_topics())
docs_lda = lda_topic_model[corpus_tf_idf]

[(0,
  '0.012*"کار" + 0.012*"استخدام" + 0.010*"پروژه" + 0.009*"بازارکار" + '
  '0.008*"منزل" + 0.008*"صورت" + 0.007*"همکاری" + 0.007*"دورکار" + '
  '0.006*"برنامه" + 0.006*"ریموت"'),
 (1,
  '0.010*"خونه" + 0.010*"کار" + 0.008*"دورکار" + 0.007*"😂" + 0.005*"خوبه" + '
  '0.005*"کرونا" + 0.004*"میشه" + 0.004*"شرکت" + 0.004*"ریموت" + '
  '0.004*"میکنه"'),
 (2,
  '0.014*"دورکار" + 0.011*"کار" + 0.008*"خونه" + 0.007*"شرکت" + 0.006*"میشه" + '
  '0.005*"ریموت" + 0.005*"حضوری" + 0.004*"😁" + 0.004*"کاری" + 0.004*"صبح"'),
 (3,
  '0.017*"چاپ" + 0.016*"منزل" + 0.014*"کار" + 0.012*"خانگی" + 0.011*"شغل" + '
  '0.009*"خانه" + 0.009*"دستگاه" + 0.008*"کرونا" + 0.008*"سیلک" + '
  '0.007*"کارمندان"')]


In [18]:
coherence_model_lda = CoherenceModel(model=lda_topic_model, texts=all_tokens, dictionary=all_dictionary, coherence='c_v')
coherence_value = coherence_model_lda.get_coherence()
coherence_value

0.37780787428720003

### Lem

In [19]:
all_tokens = token_list_lemm
all_dictionary = get_dictionary(all_tokens)
bow_corpus = get_bow_corpus(all_dictionary, all_tokens)
corpus_tf_idf = get_tf_idf_corpus(bow_corpus)
lda_topic_model = gensim.models.LdaMulticore(tqdm(corpus_tf_idf), num_topics=4, id2word=all_dictionary, passes=10, alpha=0.1)

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:04<00:00, 10205.47it/s]


In [20]:
pprint(lda_topic_model.print_topics())
docs_lda = lda_topic_model[corpus_tf_idf]

[(0,
  '0.025*"منزل" + 0.022*"کار" + 0.019*"چاپ" + 0.014*"شغل" + 0.014*"استخدام" + '
  '0.013*"خانگی" + 0.009*"دستگاه" + 0.009*"سیلک" + 0.008*"صورت" + '
  '0.007*"خانه"'),
 (1,
  '0.010*"کرونا" + 0.008*"کارمند" + 0.007*"تهران" + 0.006*"کار" + 0.006*"دولت" '
  '+ 0.005*"کارکن" + 0.005*"شرکت" + 0.005*"پروژه" + 0.005*"خانه" + '
  '0.005*"\u2063"'),
 (2,
  '0.012*"خونه" + 0.011*"کار" + 0.008*"دورکار" + 0.006*"😂" + 0.006*"شرکت" + '
  '0.006*"کاشت#کار" + 0.005*"میشه" + 0.005*"صبح" + 0.004*"مرد#میر" + '
  '0.004*"سرکار"'),
 (3,
  '0.015*"دورکار" + 0.010*"کار" + 0.007*"خونه" + 0.007*"شرکت" + 0.006*"میشه" + '
  '0.005*"ریموت" + 0.005*"خوبه" + 0.005*"کرونا" + 0.004*"گرفت#گیر" + '
  '0.004*"میکنن"')]


In [21]:
coherence_model_lda = CoherenceModel(model=lda_topic_model, texts=all_tokens, dictionary=all_dictionary, coherence='c_v')
coherence_value = coherence_model_lda.get_coherence()
coherence_value

0.35218238891271725